# 🔗 BÀI TẬP 2: SQL JOINS

## Mục tiêu học tập:
- INNER JOIN
- LEFT JOIN / RIGHT JOIN
- FULL OUTER JOIN
- CROSS JOIN
- SELF JOIN
- Multiple JOINs
- JOIN conditions và performance

In [ ]:
# Setup connection
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def get_connection():
    return psycopg2.connect(
        host='postgres',
        port=5432,
        database='data_engineer',
        user='dataengineer',
        password='dataengineer123'
    )

def run_query(query):
    conn = get_connection()
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

print("✅ Connection setup ready!")

---
## 📊 Hiểu về Database Schema

```
customers (1,000 records)
  ├── customer_id (PK)
  ├── customer_name
  ├── email
  ├── country
  └── customer_segment

orders (10,000 records)
  ├── order_id (PK)
  ├── customer_id (FK → customers)
  ├── order_date
  ├── order_status
  └── total_amount

order_items (~30,000 records)
  ├── order_item_id (PK)
  ├── order_id (FK → orders)
  ├── product_id (FK → products)
  ├── quantity
  ├── unit_price
  └── discount_percent

products (100 records)
  ├── product_id (PK)
  ├── product_name
  ├── category
  ├── price
  └── cost
```

---
## 🎯 EXERCISE 1: INNER JOIN cơ bản

**Câu hỏi:** Lấy thông tin 10 orders kèm tên customer

In [ ]:
# YOUR CODE HERE
query = """
SELECT 
    o.order_id,
    o.order_date,
    o.total_amount,
    c.customer_name,
    c.email
FROM analytics.orders o
INNER JOIN analytics.customers c ON o.customer_id = c.customer_id
LIMIT 10;
"""

result = run_query(query)
result

---
## 🎯 EXERCISE 2: LEFT JOIN

**Câu hỏi:** Tìm customers chưa có order nào (LEFT JOIN + WHERE IS NULL)

In [ ]:
# YOUR CODE HERE
query = """
-- Write your query here
-- Hint: LEFT JOIN customers với orders, WHERE order_id IS NULL
"""

result = run_query(query)
print(f"📊 Found {len(result)} customers without orders")
result.head()

---
## 🎯 EXERCISE 3: Multiple JOINs

**Câu hỏi:** Lấy thông tin order items kèm:
- Order info
- Customer name
- Product name

In [ ]:
# YOUR CODE HERE
query = """
-- Write your query here
-- Join: order_items → orders → customers
-- Join: order_items → products
"""

result = run_query(query)
result.head(10)

---
## 🎯 EXERCISE 4: Aggregation với JOIN

**Câu hỏi:** Tính tổng revenue của mỗi customer

In [ ]:
# YOUR CODE HERE
query = """
-- Write your query here
-- Hint: JOIN customers với orders, GROUP BY customer, SUM(total_amount)
"""

result = run_query(query)
result.head(10)

---
## 🎯 EXERCISE 5: SELF JOIN

**Câu hỏi:** Tìm customers ở cùng country (SELF JOIN)

In [ ]:
# YOUR CODE HERE
query = """
-- Write your query here
-- Hint: JOIN customers c1 với customers c2 ON c1.country = c2.country
-- WHERE c1.customer_id < c2.customer_id (để tránh duplicate)
"""

result = run_query(query)
result.head(10)

---
## 🎯 EXERCISE 6: Category hierarchy JOIN

**Câu hỏi:** Hiển thị category hierarchy (parent-child relationship)

In [ ]:
# YOUR CODE HERE
query = """
-- Write your query here
-- Hint: SELF JOIN categories với parent_category_id
"""

result = run_query(query)
result

---
## 🎯 CHALLENGE 1: Top customers by category

**Câu hỏi:** Tìm top 5 customers chi tiêu nhiều nhất cho mỗi product category

In [ ]:
# YOUR CODE HERE
query = """
-- Write your query here
-- Hint: Multiple JOINs + GROUP BY + Window function hoặc subquery
"""

result = run_query(query)
result

---
## 🎯 CHALLENGE 2: Customer purchase pattern

**Câu hỏi:** Phân tích:
- Tổng số orders
- Tổng revenue
- Average order value
- Số lượng products khác nhau đã mua
- Categories đã mua

Cho mỗi customer

In [ ]:
# YOUR CODE HERE
query = """
-- Write your query here
"""

result = run_query(query)
result.head(10)

---
## 📊 VISUALIZATION: Top customers

Visualize top 10 customers by revenue

In [ ]:
query = """
SELECT 
    c.customer_name,
    COUNT(o.order_id) as total_orders,
    SUM(o.total_amount) as total_revenue
FROM analytics.customers c
INNER JOIN analytics.orders o ON c.customer_id = o.customer_id
WHERE o.order_status = 'completed'
GROUP BY c.customer_id, c.customer_name
ORDER BY total_revenue DESC
LIMIT 10;
"""

df = run_query(query)

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Revenue chart
ax1.barh(df['customer_name'], df['total_revenue'])
ax1.set_xlabel('Total Revenue ($)')
ax1.set_title('Top 10 Customers by Revenue')
ax1.invert_yaxis()

# Orders chart
ax2.barh(df['customer_name'], df['total_orders'], color='orange')
ax2.set_xlabel('Total Orders')
ax2.set_title('Top 10 Customers by Order Count')
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

---
## 📚 SOLUTIONS

In [ ]:
# # SOLUTION 2: LEFT JOIN - Customers without orders
# query = """
# SELECT 
#     c.customer_id,
#     c.customer_name,
#     c.email,
#     c.signup_date
# FROM analytics.customers c
# LEFT JOIN analytics.orders o ON c.customer_id = o.customer_id
# WHERE o.order_id IS NULL;
# """

# # SOLUTION 3: Multiple JOINs
# query = """
# SELECT 
#     oi.order_item_id,
#     o.order_id,
#     o.order_date,
#     c.customer_name,
#     p.product_name,
#     p.category,
#     oi.quantity,
#     oi.unit_price,
#     oi.line_total
# FROM analytics.order_items oi
# INNER JOIN analytics.orders o ON oi.order_id = o.order_id
# INNER JOIN analytics.customers c ON o.customer_id = c.customer_id
# INNER JOIN analytics.products p ON oi.product_id = p.product_id
# LIMIT 20;
# """

# # SOLUTION 4: Aggregation với JOIN
# query = """
# SELECT 
#     c.customer_id,
#     c.customer_name,
#     c.customer_segment,
#     COUNT(o.order_id) as total_orders,
#     SUM(o.total_amount) as total_revenue,
#     AVG(o.total_amount) as avg_order_value
# FROM analytics.customers c
# LEFT JOIN analytics.orders o ON c.customer_id = o.customer_id
# GROUP BY c.customer_id, c.customer_name, c.customer_segment
# ORDER BY total_revenue DESC NULLS LAST
# LIMIT 20;
# """

# # SOLUTION 5: SELF JOIN
# query = """
# SELECT 
#     c1.customer_name as customer_1,
#     c2.customer_name as customer_2,
#     c1.country
# FROM analytics.customers c1
# INNER JOIN analytics.customers c2 
#     ON c1.country = c2.country 
#     AND c1.customer_id < c2.customer_id
# WHERE c1.country = 'Vietnam'
# LIMIT 20;
# """

# # SOLUTION 6: Category hierarchy
# query = """
# SELECT 
#     c1.category_id,
#     c1.category_name as child_category,
#     c2.category_name as parent_category
# FROM analytics.categories c1
# LEFT JOIN analytics.categories c2 ON c1.parent_category_id = c2.category_id
# ORDER BY c2.category_name, c1.category_name;
# """